## import

In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

## data

In [11]:
ratings = pd.read_csv('../processed/FM_classifciation 3.csv')
ratings

,country_id,year,item_id,preference,GDP_growth,pop_growth,diff
0,Afghanistan,2013,16.0,6.321578e-07,0.101959,0.287706,0
1,Afghanistan,2013,19.0,1.799218e-06,0.101959,0.287706,1
2,Afghanistan,2013,21.0,6.905108e-06,0.101959,0.287706,0
3,Afghanistan,2013,27.0,2.577259e-06,0.101959,0.287706,0
4,Afghanistan,2013,28.0,1.458826e-06,0.101959,0.287706,1
...,...,...,...,...,...,...,...
376526,Zimbabwe,2021,82.0,2.467287e-07,0.132152,0.233088,1
376527,Zimbabwe,2021,85.0,3.288542e-05,0.132152,0.233088,1
376528,Zimbabwe,2021,87.0,4.543334e-05,0.132152,0.233088,1
376529,Zimbabwe,2021,90.0,1.134952e-05,0.132152,0.233088,0


## encoding

In [12]:
# year encoding
year_dict = {}
for i in set(ratings['year']):
    year_dict[i] = len(year_dict)
n_year = len(year_dict)                  

# Country encoding
country_dict = {}
start_point = n_year                     
for i in set(ratings['country_id']):
    country_dict[i] = len(country_dict)
n_country = len(country_dict)            
start_point += n_country                 

# Item encoding
item_dict = {}
start_point = n_country                  
for i in set(ratings['item_id']):
    item_dict[i] = start_point + len(item_dict)
n_item = len(item_dict)                  
start_point += n_item                    

# preference encoding
pre_index = start_point
start_point += 1

# GDP_growth encoding   
gdp_index = start_point
start_point += 1

# pop_growth encoding
pop_index = start_point
start_point += 1
num_x = start_point                     # 전체 변수의 숫자 (년도 + 국가 개수 + 품목 개수 + GDP_growth + pop_growth, preference) 총 311개

In [13]:
x = ratings

In [14]:
# Generate X data
data = []
y = []
for i in range(len(x)):
    case = x.iloc[i]
    x_index = []
    x_value = []
    x_index.append(year_dict[case['year']])        # year id one-hot encoding
    x_value.append(1.)                             # 있으면 1
    x_index.append(country_dict[case['country_id']])  # country id one-hot encoding
    x_value.append(1.)
    x_index.append(item_dict[case['item_id']])     # item id one-hot encoding
    x_value.append(1.)
    x_index.append(pre_index)                      # preference
    x_value.append(case['preference'])
    x_index.append(gdp_index)                      
    x_value.append(case['GDP_growth'])
    x_index.append(pop_index)
    x_value.append(case['pop_growth'])
    data.append([x_index, x_value])                # 만들어진 한줄의 데이터를([[year_index, country_index, item_index, gdp_index, pop_index], [1, 1, 1, 1, 1]] 형태) data에 저장
    y.append(case['diff'])                         # 종속변수
    if (i % 30000) == 0:
        print('Encoding ', i, ' cases...')

Encoding  0  cases...
Encoding  30000  cases...
Encoding  60000  cases...
Encoding  90000  cases...
Encoding  120000  cases...
Encoding  150000  cases...
Encoding  180000  cases...
Encoding  210000  cases...
Encoding  240000  cases...
Encoding  270000  cases...
Encoding  300000  cases...
Encoding  330000  cases...
Encoding  360000  cases...


In [7]:
### Train-Test split
trainset = data[:145693]
testset = data[145693:]   # 2017년 데이터
print(len(trainset), len(testset))

145693 38975


In [8]:
# y데이터에서 2017년 데이터 지우기
y = y[:145693]
print(len(y))

145693


## model

In [9]:
class FactorizationMachine():
    def __init__(self, N, K, data, y, test_data, lr, l2_reg, l2_lambda, epoch, early_stop_window, train_ratio=0.75):
        """
        :param k: number of latent vector
        :param lr: learning rate
        :param l2_reg: bool parameter for L2 regularization
        :param l2_lambda: lambda of L2 regularization
        :param epoch: training epoch
        """
        self._K = K
        self._N = N
        self._data = data
        self._y = y
        self._test_data = test_data
        self._lr = lr
        self._l2_reg = l2_reg
        self._l2_lambda = l2_lambda
        self._epoch = epoch
        self._early_stop_window = early_stop_window
        self._valid_loss_list = []
        # Train/Valid/Test 분리
        cutoff = int(train_ratio * len(data))
        self.train_x = data[:cutoff]
        self.valid_x = data[cutoff:]
        self.train_y = y[:cutoff]
        self.valid_y = y[cutoff:]
        self.test_x = test_data

    def _load_dataset(self):
        # init FM vectors
        self._init_vectors()
        print("Finish init FM vectors.")

    def _init_vectors(self):
        # w 초기화
        self.w = np.random.normal(scale=1./self._N, size=(self._N))          # 사이즈는 변수의 수 만큼
        # v 초기화
        self.v = np.random.normal(scale=1./self._K, size=(self._N, self._K))  # 사이즈는 (변수의 수 X K)
        self.train_x_data = []
        self.train_y_data = np.zeros((len(self.train_x)))
        self.valid_x_data = []
        self.valid_y_data = np.zeros((len(self.valid_x)))

        self.train_y_data = np.array(self.train_y)
        for n, row in enumerate(self.train_x):
            self.train_x_data.append([np.array(row[0]), np.array(row[1])])
        
        self.valid_y_data = np.array(self.valid_y)
        for n, row in enumerate(self.valid_x):
            self.valid_x_data.append([np.array(row[0]), np.array(row[1])])

    def train(self):
        """
        Train FM model by Gradient Descent with L2 regularization
        """
        self._load_dataset()
        for epoch_num in range(1, self._epoch):
            train_y_hat = self.predict(data=self.train_x_data)
            valid_y_hat = self.predict(data=self.valid_x_data)
            train_loss = self._get_loss(y_data=self.train_y_data, y_hat=train_y_hat)
            valid_loss = self._get_loss(y_data=self.valid_y_data, y_hat=valid_y_hat)
            train_auc = roc_auc_score(self.train_y_data, train_y_hat)
            valid_auc = roc_auc_score(self.valid_y_data, valid_y_hat)
            self._print_learning_info(epoch=epoch_num, train_loss=train_loss, valid_loss=valid_loss,
                                      train_auc=train_auc, valid_auc=valid_auc)
            if self._check_early_stop(valid_loss=valid_loss):
                print("Early stop at epoch:", epoch_num)
                return 0

            self._stochastic_gradient_descent(self.train_x_data, self.train_y_data)

    def predict(self, data):
        """
        Implementation of FM model's equation on O(kmd)

        -----
        Numpy array shape : (n, [index of md], [value of md])
        md : none-zero feature
        """  
        num_data = len(data)
        scores = np.zeros(num_data)
        for n in range(num_data):
            feat_idx = data[n][0]
            val = data[n][1]

            # linear feature score
            linear_feature_score = np.sum(self.w[feat_idx] * val)

            # factorized feature score
            vx = self.v[feat_idx] * (val.reshape(-1, 1))
            cross_sum = np.sum(vx, axis=0)
            square_sum = np.sum(vx * vx, axis=0)
            cross_feature_score = 0.5 * np.sum(np.square(cross_sum) - square_sum)

            # Model's equation
            scores[n] = linear_feature_score + cross_feature_score

        # Sigmoid transformation for binary classification
        scores = 1.0 / (1.0 + np.exp(-scores))
        return scores

    def inference(self, data):
        
        self.test_x_data = []
        for n, row in enumerate(self.test_x):
            self.test_x_data.append([np.array(row[0]), np.array(row[1])])

        num_data = len(data)
        scores = np.zeros(num_data)
        for n in range(num_data):
            feat_idx = data[n][0]
            val = np.array(data[n][1])

            # linear feature score
            linear_feature_score = np.sum(self.w[feat_idx] * val)

            # factorized feature score
            vx = self.v[feat_idx] * (val.reshape(-1, 1))
            cross_sum = np.sum(vx, axis=0)
            square_sum = np.sum(vx * vx, axis=0)
            cross_feature_score = 0.5 * np.sum(np.square(cross_sum) - square_sum)

            # Model's equation
            scores[n] = linear_feature_score + cross_feature_score

        # Sigmoid transformation for binary classification
        scores = 1.0 / (1.0 + np.exp(-scores))
        print(scores)
        return scores

    def _get_loss(self, y_data, y_hat):
        """
        Calculate loss with L2 regularization (two type of coeficient - w,v)
        """
        l2_norm = 0
        if self._l2_reg:
            w_norm = np.sqrt(np.sum(np.square(self.w)))
            v_norm = np.sqrt(np.sum(np.square(self.v)))
            l2_norm = self._l2_lambda * (w_norm + v_norm)
        return -1 * np.sum( (y_data * np.log(y_hat)) + ((1 - y_data) * np.log(1 - y_hat)) ) + l2_norm

    def _check_early_stop(self, valid_loss):
        self._valid_loss_list.append(valid_loss)
        if len(self._valid_loss_list) > 5:
            prev_loss = self._valid_loss_list[len(self._valid_loss_list) - self._early_stop_window]
            curr_loss = valid_loss
            if prev_loss < curr_loss:
                return True
        return False

    def _print_learning_info(self, epoch, train_loss, valid_loss, train_auc, valid_auc):
        print("epoch:", epoch, "||", "train_loss:", train_loss, "||", "valid_loss:", valid_loss,
              "||", "Train AUC:", train_auc, "||", "Test AUC:", valid_auc)

    def _stochastic_gradient_descent(self, x_data, y_data):
        """
        Update each coefs (w, v) by Gradient Descent
        """
        for data, y in zip(x_data, y_data):
            feat_idx = data[0]
            val = data[1]
            vx = self.v[feat_idx] * (val.reshape(-1, 1))

            # linear feature score
            linear_feature_score = np.sum(self.w[feat_idx] * val)

            # factorized feature score
            vx = self.v[feat_idx] * (val.reshape(-1, 1))
            cross_sum = np.sum(vx, axis=0)
            square_sum = np.sum(vx * vx, axis=0)
            cross_feature_score = 0.5 * np.sum(np.square(cross_sum) - square_sum)

            # Model's equation
            score = linear_feature_score + cross_feature_score
            y_hat = 1.0 / (1.0 + np.exp(-score))
            cost = y_hat - y

            if self._l2_reg:
                self.w[feat_idx] = self.w[feat_idx] - cost * self._lr * (val + self._l2_lambda * self.w[feat_idx])
                self.v[feat_idx] = self.v[feat_idx] - cost * self._lr * ((sum(vx) * (val.reshape(-1, 1)) - (vx * (val.reshape(-1, 1)))) + self._l2_lambda * self.v[feat_idx])
            else:
                self.w[feat_idx] = self.w[feat_idx] - cost * self._lr * val
                self.v[feat_idx] = self.v[feat_idx] - cost * self._lr * (sum(vx) * (val.reshape(-1, 1)) - (vx * (val.reshape(-1, 1))))

In [10]:
if __name__ == "__main__":
    fm = FactorizationMachine(N = num_x,
                              K=4,
                              data = trainset,
                              y = y,
                              test_data = testset,
                              lr=0.001,
                              l2_reg=True,
                              l2_lambda=0.0001,
                              epoch=200,
                              early_stop_window=3)

    fm.train()
    predict_proba = fm.inference(data=testset)

Finish init FM vectors.
epoch: 1 || train_loss: 75935.69517890604 || valid_loss: 25658.310164676364 || Train AUC: 0.5133722167803325 || Test AUC: 0.505717204027838
epoch: 2 || train_loss: 68612.78989631005 || valid_loss: 23219.077499006704 || Train AUC: 0.6142979765339635 || Test AUC: 0.5710439087939472
epoch: 3 || train_loss: 67394.20125948155 || valid_loss: 22751.17992477818 || Train AUC: 0.6437986217536693 || Test AUC: 0.597903303132004
epoch: 4 || train_loss: 66747.64656017786 || valid_loss: 22550.799821060016 || Train AUC: 0.656323374743282 || Test AUC: 0.6091990010103477
epoch: 5 || train_loss: 66346.46619872343 || valid_loss: 22449.606174769204 || Train AUC: 0.6630126400428296 || Test AUC: 0.6151947649255572
epoch: 6 || train_loss: 66072.65938320672 || valid_loss: 22389.70231661666 || Train AUC: 0.6673985140578118 || Test AUC: 0.6189080706102865
epoch: 7 || train_loss: 65871.90404772111 || valid_loss: 22350.315160652714 || Train AUC: 0.6705460527237596 || Test AUC: 0.62135318123

## result

In [11]:
result = []
for i in range(len(predict_proba)):
    if predict_proba[i] >= 0.5: #상승
        result.append(1)
    else:
        result.append(0)

In [14]:
import pickle
with open('./processed/predict_results2013_2016.pickle', 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)